In [ ]:
pip install tensorflow_text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 29.0 MB/s eta 0:00:00


Study design

In [ ]:
import openpyxl
import tensorflow_hub as hub
import tensorflow_text

# Load the USE model
model_url = "https://tfhub.dev/google/universal-sentence-encoder-multilingual/3"
embed = hub.load(model_url)

# Define the categories
categories = ["Cross-sectional study", "Case-control study", "Experimental study", "Quasi-experimental study", 
              "Randomized controlled trial (RCT)", "Ecological study", "Case series and case reports", "Systematic review", 
              "Meta-analysis",  "Correlational study", "Descriptive study",  "Qualitative study", 
              "Mixed methods study", "Non-randomized trial", 
              "Content analysis", "Feasibility study",  "Action research study",  "Historical research study", 
             "Meta-synthesis", "Prospective cohort study", "Retrospective cohort study",
               "Mendelian randomization study", "Network meta-analysis", "Diagnostic acuracy study", "In-vivo study", 
              "In-vitro study", "In-silico"]
# Open the Excel file and select the first worksheet
wb = openpyxl.load_workbook('study.xlsx')
ws = wb.active

# Get the maximum number of rows in the Excel file
max_row = ws.max_row

# Loop over each row in the Excel file
for i in range(1, max_row+1):
    # Get the input sentence from the first column of the current row
    input_sentence = ws.cell(row=i, column=1).value
    
    # Embed the input sentence using USE
    embeddings = embed([input_sentence])
    
    # Calculate the similarity between the input sentence and each category using cosine similarity
    similarity_scores = embeddings.numpy().dot(embed(categories).numpy().T)
    
    # Find the category with the highest similarity score
    predicted_category = categories[similarity_scores.argmax()]
    
    # Save the predicted category in the second column of the current row
    ws.cell(row=i, column=2).value = predicted_category

# Save the modified Excel file
wb.save('output_file.xlsx')

Disease

In [ ]:
pip install xlwt tensorflow_text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.0/100.0 kB 3.8 MB/s eta 0:00:00


disease/ICD/MESH

In [ ]:
import pandas as pd
import tensorflow_hub as hub
import tensorflow_text

# Load the USE model
model_url = "https://tfhub.dev/google/universal-sentence-encoder-multilingual/3"
embed = hub.load(model_url)

# Read the category Excel file
category_df = pd.read_excel("diseases.xlsx")

# Get the categories from the first column
categories = category_df.iloc[:, 0].tolist()

# Read the input Excel file
input_df = pd.read_excel("study.xlsx", header=None)

# Define the name of the new column to save the results
new_column_name = "Category"

# Set a threshold for the similarity score
similarity_threshold = 0.5

# Loop through each row in the first column and classify the sentences
for i, input_sentence in input_df.iloc[:, 0].iteritems():
    try:
        embeddings = embed([input_sentence])
        similarity_scores = embeddings.numpy().dot(embed(categories).numpy().T)
        max_similarity_score = similarity_scores.max()
        if max_similarity_score >= similarity_threshold:
            predicted_category = categories[similarity_scores.argmax()]
        else:
            predicted_category = input_sentence
        input_df.at[i, new_column_name] = predicted_category
    except Exception as e:
        print(f"Error: {e}")
        input_df.at[i, new_column_name] = "Error"

# Save the results to the input Excel file
input_df.to_excel("test.xls", index=False)


<ipython-input-5-1c64a06cd3c6>:25: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for i, input_sentence in input_df.iloc[:, 0].iteritems():


KeyboardInterrupt: ignored